In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
gau_rages_page = requests.get("https://finalfantasy.wikia.com/wiki/Rage_(Final_Fantasy_VI_command)")
soup = BeautifulSoup(gau_rages_page.content, "html.parser")

In [4]:
list_of_rages_header = soup.find("span", id="List_of_Rages")
rage_table = list_of_rages_header.findNext("table")

In [ ]:
all_table_th = rage_table.findAll("th")
base_link = "https://finalfantasy.wikia.com"
ff6_url = "/wiki/Final_Fantasy_VI"
enemy_abilities_link = "https://finalfantasy.wikia.com/wiki/List_of_Final_Fantasy_VI_enemy_abilities"

abilities = requests.get(enemy_abilities_link)
abilities_soup = BeautifulSoup(abilities.content, "html.parser")

# For specific spells, do certain crawling exceptions
specific_exceptions = set()
specific_exceptions.add("Blizzaga_(ability)")

misspells_dict = {}
misspells_dict["Incisors"] = "Incisor"
misspells_dict["Landslide"] = "Land_Slide"

visited_id = set()
# Skip these specific spells
visited_id.add("Tornado_(ability)")
visited_id.add("Break_(ability)")

output_file = "spell_power.csv"
headers = ["Ability Name", "Magic Power", "Description"]

df = pd.DataFrame(columns=headers)
    
for i in range(7, len(all_table_th), 3):
    cur_th = all_table_th[i]
    a_link = cur_th.find("a").get('href')
    ability_link = "".join((base_link, a_link))

    # For now, only check the enemy abilities link
    if (enemy_abilities_link in ability_link):
        id_1 = ability_link.split("#")[1]
        # If misspelled
        if id_1 in misspells_dict:
            id_1 = misspells_dict[id_1]

        next_row = []
        if id_1 not in visited_id:
            visited_id.add(id_1)

            span_found = abilities_soup.find("span", id=id_1)
            # If we cannot find the span, we need to manually add it to misspells_dict
            if span_found is None:
                print(ability_link)

            next_td = str(span_found.findNext("td"))
            # if the next td has the word "spell power" in it
            spell_power = 0
            physical_power = 0

            if "spell power" in next_td.lower():
                index_of_spell = next_td.index("Spell Power")
                substr_1 = next_td[index_of_spell - 10:index_of_spell + 25]
                temp_power = []
                # Iterate through substr until you find a digit
                for char in substr_1:
                    if char.isdigit():
                        temp_power.append(char)
                temp_power = int("".join(temp_power))

                next_row.append(id_1)
                next_row.append(temp_power)
                next_row.append(next_td)
                df.loc[len(df.index)] = next_row
                
                s = ",".join((str(id_1), str(temp_power), next_td, "\n"))
#                 file.write(s)
    
    else:
        # Not on the "ability page" page
        temp_page = requests.get(ability_link)
        temp_soup = BeautifulSoup(temp_page.content, "html.parser")
        temp_ff6 = temp_soup.find("a", href=ff6_url, title="Final Fantasy VI")
        
        if temp_ff6 is not None:
            id_1 = ability_link.split("wiki/")[1]
            
            if id_1 not in visited_id:
                visited_id.add(id_1)
                
                if id_1 in specific_exceptions:
                    temp_power = 122
                    p = "<p>Blizzaga is an <a href=\"/wiki/Magic_(Final_Fantasy_VI_command)\" title=\"Magic (Final Fantasy VI \
                         command)\">Attack spell</a> thought by <a href=\"/wiki/Valigarmanda\" title=\"Valigarmanda\">\
                         Valigarmanda</a> at a rate of x1. <a href=\"/wiki/Celes_Chere/Gameplay\" title=\"Celes Chere\
                         /Gameplay\">Celes</a> also learns the spell at level 42. It costs 52 MP to cast, has a hit rate \
                         of 150, an attack power of 122, and is vulnerable to <a href=\"/wiki/Runic\" \
                         title=\"Runic\">Runic</a>. The spell is used by many enemies, the earliest being used by \
                         <a href=\"/wiki/Dullahan_(Final_Fantasy_VI)\" title=\"Dullahan (Final Fantasy VI)\">Dullahan</a>.</p>"
                    next_row.append(id_1)
                    next_row.append(temp_power)
                    next_row.append(p)
                    df.loc[len(df.index)] = next_row
            
                else:
                    # Find the next 3 <p> tags
                    next_p_1 = temp_ff6.find_next("p")
                    next_p_2 = next_p_1.find_next("p")
                    next_p_3 = next_p_2.find_next("p")
                    next_p_list = [next_p_1, next_p_2, next_p_3]

                    next_row = []
                    for p in next_p_list:
                        p = str(p)
                        if "power " in str(p).lower():
                            index_of_power = p.lower().index("power ")
                            # substr_1 = p[index_of_power - 10: index_of_power + 25]
                            substr_1 = p[index_of_power: index_of_power + 25]
                            temp_power = []

                            # Iterate through substr until you find a digit
                            for char in substr_1:
                                if char.isdigit():
                                    temp_power.append(char)
                                    if (len(temp_power)) > 2:
                                        break
                            try:
                                temp_power = int("".join(temp_power))
                            except ValueError:
                                print(df.to_string())
                                print("{}, {}".format(p, substr_1))
                                raise
                            next_row.append(id_1)
                            next_row.append(temp_power)
                            next_row.append(p)
                            df.loc[len(df.index)] = next_row
                            break
                
        # Weird links
        else:
            print("INVALID LINK: {}".format(ability_link))    

In [ ]:
# test_url = "https://finalfantasy.wikia.com/wiki/Fira_(ability)#Final_Fantasy_VI"
# ff6_url = "/wiki/Final_Fantasy_VI"
# page = requests.get(test_url)
# delicious_soup = BeautifulSoup(page.content, "html.parser")

# temp_ff6 = delicious_soup.find("a", href=ff6_url, title="Final Fantasy VI")
# p_1 = temp_ff6.find_next("p")
# p_2 = p_1.find_next("p")
# p_3 = p_2.find_next("p")
# # print(p_1)
# # print(p_2)
# # print(p_3)
# list1 = [p_1, p_2, p_3]

# for p in list1:
#     p = str(p)
#     if "power" in str(p):
#         index_of_power = p.index("power")
#         substr_1 = p[index_of_power - 10:index_of_power + 25]
#         print(substr_1)
#         break

df2 = df.sort_values("Magic Power", ascending=False)
df2

In [ ]:
df2.to_csv(output_file)

In [ ]:
url = "https://finalfantasy.wikia.com/wiki/Blizzaga_(ability)#Final_Fantasy_VI"
r = requests.get(url)
bs = BeautifulSoup(r.content, "html.parser")
bs_ff6 = bs.find("a", href=ff6_url, title="Final Fantasy VI")
b = bs_ff6.find_all_next("p")

for b1 in b:
    f = b1.find("a")
    href1 = f.get("href")
    if "Final_Fantasy_VI" in href1:
        print(b1)
        break

# Get Checkbox HTML String

In [67]:
def get_checkboxes(value, name):
    """
    Return a bootstrap checkbox.
    """
#     <div class="col-md-3 form-check form-check-inline">
#       <input class="form-check-input" type="checkbox" id="inlineCheckbox1" value="option1">
#       <label class="form-check-label" for="inlineCheckbox1">1</label>
#     </div>
    max_value_size = 27
    if (len(value) > max_value_size + 3):
        value = value[:max_value_size]
        value = "".join((value, "..."))
    
    id_1 = value.replace(" ", "_")
    
    s0 = "<div class=\"form-check-inline col-md-3\">"
    s1 = "<input class=\"form-check-input\" type=\"checkbox\" id=\"{}\" value=\"{}\" name=\"{}\" \\>".format(id_1, value, name)
    s2 = "<label class=\"form-check-label\" for=\"{}\">{}</label>".format(id_1, value)
    s3 = "</div>"
    
    s3 = "".join((s0, s1, s2, s3))
    return s3


def add_calculate_button(id_1):
    """
    Add a calculate button
    """
    s = "<div class=\"form-row customPaddingBottom\"><button type=\"button\" id=\"{}\" class=\"btn btn-primary\">Calculate!</button></div>".format(id_1)
    return s


def get_output_textinputs(total_num_groups):
    """
    Add textinputs that are used as outputs.
    
    # ARGUMENTS
    total_num_groups -> total number of groups.
    """
    # <div class="form-row">
    #   <div class="col-md-1">
    #     <label for="group0Output">Group 0</label>
    #     <input type="text" class="form-control" id="group0Output"/>
    #   </div>
    #   <div class="col-md-1">
    #     <label for="group0Output">Group 0</label>
    #     <input type="text" class="form-control" id="group0Output"/>
    #   </div>
    # </div>
    s = ""
    every_8 = "</div><div class=\"form-row customPaddingBottom\">"
    
    for i in range(total_num_groups + 1):
        if i % 8 == 0:
            s = "".join((s, every_8))
        group_id = "group{}Output".format(i)
        group_name = "Group {}".format(i)
        # If medium-sized screen or above, use 1 column
        # Smallest size-screen, use 4 columns
        html_str = "<div class=\"col-md-1 col-4\"><label for=\"{}\">{}</label><input type=\"text\" class=\"form-control\" id=\"{}\"/></div>".format(group_id, group_name, group_id)
        s = "".join((s, html_str))
    
    one_div = "</div>"
    s = s[len(one_div):]
    s = "".join((s, one_div))
    s = "".join(("<div id=\"checkboxHexOutputs\"", s, "</div>"))
    
    return s


all_table_th = rage_table.findAll("th")
count = 0
cur_group = 0
group_name = "group{}".format(cur_group)

starting_div_every8_base = "<div class=\"customPaddingBottom form-row\""
cur_html_str = ""
every8_len = 0


for i in range(5, len(all_table_th), 3):    
    # Get current monster's name
    cur_th = all_table_th[i].text.strip()
    group_name = ""
    
    if count % 8 == 0:
        if count != 0:
            cur_group += 1
            
        group_name = "group{}".format(cur_group)
        # e.g., id="group0Div"
        id_8 = "".join((group_name, "Div"))
        id_8_wholestr = "id=\"{}\"".format(id_8)
        starting_div_every8 = "".join((starting_div_every8_base, id_8_wholestr, ">"))
        group_label = "".join(("Group ", str(cur_group)))
        label_8 = "<div class=\"col-md-12\"><label for=\"{}\"><strong>{}</strong></label></div>".format(id_8, group_label)
        
        cur_html_str = "".join((cur_html_str, "</div>", starting_div_every8, label_8))
        if every8_len == 0:
            every8_len = len(starting_div_every8) + len(label_8)
        
    html_str = get_checkboxes(cur_th, group_name)
    cur_html_str = "".join((cur_html_str, html_str))
    
    count += 1

# Remove the first </div>
first_closing_divs = "</div>"
index_start = len(first_closing_divs)
cur_html_str = cur_html_str[index_start:]
# Add in 2 </div>
cur_html_str = "".join((cur_html_str, first_closing_divs))

calc_button = add_calculate_button(id_1="gauRageButton")
output_textinputs = get_output_textinputs(total_num_groups=cur_group)

cur_html_str = "".join((cur_html_str, calc_button, output_textinputs))


with open("output.txt", "w") as file:
    file.write(cur_html_str)
    
import os
os.system("start " + "output.txt")

0